# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy
import pickle

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('./test_imgs/dep.p', 'rb') as dep:
    d = pickle.load(dep, encoding="16UC1")

In [3]:
with open('./test_imgs/rgb.p', 'rb') as rgb:
    r = pickle.load(rgb)


In [4]:
d.max()

915

## Read Test Images

In [5]:
# img_color = cv2.imread('test_imgs/jenga_tower_color.png')
# img_depth = cv2.imread('test_imgs/jenga_tower_depth.png', 0)
# img_color = cv2.imread('/home/shs/Desktop/rgb1.png')
# img_depth = cv2.imread('/home/shs/Desktop/dep.png', 0)
img_color = r
img_depth = d
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

cv2.imshow(f'color', img_color)
# # cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 

color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


In [75]:
type(img_hsv)

numpy.ndarray

## Extract Blocks' Masks By Colors

In [6]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [7]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [8]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 55, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,55,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 60, 60)
upper_violet = (130+20, 255, 255)

In [9]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2
        
        # Denoise by Erosion and Dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=1)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=1)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    # if color == 'violet':
    #     cv2.imshow('src_bin', src_bin)
    #     # cv2.imwrite('../imgs/red_mask.png', src_bin)
    #     cv2.imshow(f'{color}_filtered', each_color_filtered)
    #     # cv2.imwrite('../imgs/red_extract.png', each_color_filtered)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 300:
            continue
        
        # if color == 'violet':
        #     cv2.imshow('blk clr', block_color)
        #     # cv2.imwrite('../imgs/red_block2_color.png', block_color)
        #     cv2.imshow('blk msk', block_mask)
        #     # cv2.imwrite('../imgs/red_block2_mask.png', block_mask)
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()
            
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [10]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [11]:
# cv2.imshow('tower mask', tower_mask)
# # cv2.imwrite('../imgs/tower_mask.png', tower_mask)
# cv2.imshow('tower color', tower_color)
# # cv2.imwrite('../imgs/tower_color.png', tower_color)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
6 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [13]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[968.813, 0, 1023.83],
                              [0, 968.635, 775.975],
                              [0, 0, 1]]


# extrinsic = np.array([[0.999987, 0.00512943, 0.000573447, -32.0559],
#                      [-0.00516119, 0.994725, 0.102443, -1.91788],
#                      [-4.49478e-05, -0.102445, 0.994739, 3.99916],
#                      [0, 0, 0, 1]])
# extrinsic

In [14]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])
mesh_frame_small = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.1, origin=[0, 0, 0])

In [15]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    # rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, depth_scale=1, depth_trunc=3000.0, convert_rgb_to_intensity=False)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image, convert_rgb_to_intensity=False)
    # print(np.asarray(rgbd_image.depth).max())
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [16]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [17]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)


In [72]:
print(type(tower_pcd))

<class 'open3d.cpu.pybind.geometry.PointCloud'>


In [18]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 269656 points.
[[-0.01771596 -0.07790306  0.215     ]
 [-0.0175754  -0.0782654   0.21600001]
 [-0.01735245 -0.0782654   0.21600001]
 ...
 [-0.01776494  0.10756967  0.23      ]
 [-0.01752753  0.10756967  0.23      ]
 [-0.01729013  0.10756967  0.23      ]]


In [19]:
points = np.asarray(tower_pcd.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -0.19102764527657992 0.05288281771218741
Y min/max: -0.31632264375882324 0.11035418559026333
Z min/max: 0.2150000035762787 0.8889999985694885


In [20]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    # if color != 'blue':
    #     continue
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(1024, 0.025)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='blue':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [21]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

PointCloud with 268886 points.


In [22]:
points = np.asarray(pcd_combined.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -0.06142136013038187 0.05288281771218741
Y min/max: -0.08187781871597376 0.11035418559026333
Z min/max: 0.2150000035762787 0.3070000112056732


In [23]:
np.max(points[:, 1]) - np.min(points[:, 1])

0.1922320043062371

In [24]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [25]:

# o3d.visualization.draw_geometries([pcd_combined, mesh_frame_small])

# o3d.visualization.draw_geometries([pcd_combined, mesh_frame])

In [26]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy_m.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()

pcd_mesh = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_mesh)
# o3d.visualization.draw_geometries([pcd_mesh, mesh_frame_small])

TriangleMesh with 12 points and 4 triangles.
PointCloud with 268886 points.


In [27]:
points = np.asarray(pcd_mesh.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -0.037499760832788435 0.037500001490116126
Y min/max: -0.03749975206107941 0.037500001490116126
Z min/max: 5.928331776496703e-07 0.17999949160840048


In [28]:
initial_transform = np.asarray([[0, 0, -1, 0],
                                [1, 0, 0, 0],
                                [0, -1, 0, 0],
                                [0, 0, 0, 1]])

In [29]:
def prepare_icp(source, target):
    source_tmp = copy.deepcopy(source)
    target_tmp = copy.deepcopy(target)
    
    # make the point cloud into right position
    source_tmp.transform(initial_transform)
    
    # move the source pcd to do icp
    move = np.array(target_tmp.get_oriented_bounding_box().get_center() - source_tmp.get_oriented_bounding_box().get_center())

    source_tmp.transform(np.linalg.inv(initial_transform))
    
    o3d.visualization.draw_geometries([source_tmp, target_tmp, mesh_frame_small])
    
    return source_tmp, target_tmp, move

In [30]:
source, target,  move = prepare_icp(pcd_combined, pcd_mesh)

In [31]:
move

array([0.27372625, 0.03336021, 0.103793  ])

In [32]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    # source_temp.paint_uniform_color([1, 0.706, 0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, mesh_frame_small])

In [33]:
threshold = 10
trans_init = np.asarray([[0, 0, -1, move[0]],
                         [1, 0, 0, move[1]],
                         [0, -1, 0, move[2]],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

In [34]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.507357e-02, and correspondence_set size of 268886
Access transformation to get result.


In [35]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.539244e-03, and correspondence_set size of 268886
Access transformation to get result.
Transformation is:
[[-0.85940747  0.01988883 -0.51090433  0.14084215]
 [ 0.51071093 -0.01420518 -0.85963513  0.24441646]
 [-0.02435463 -0.99970128  0.00205059  0.10190226]
 [ 0.          0.          0.          1.        ]]


In [36]:
draw_registration_result(source, target, reg_p2p.transformation)

In [37]:
source_temp = copy.deepcopy(source)

source_temp.transform(reg_p2p.transformation)


PointCloud with 268886 points.

In [38]:
(np.array(source_temp.points))[:,2].min()

-0.007397777558511842

## Blocks Transformation

In [39]:
trans1 = reg_p2p.transformation

In [40]:
trans = copy.deepcopy(reg_p2p.transformation)

In [41]:
print(trans)
# trans[0,3]/=1000
# trans[1,3]/=1000
# trans[2,3]/=1000

print(np.linalg.inv(trans))

[[-0.85940747  0.01988883 -0.51090433  0.14084215]
 [ 0.51071093 -0.01420518 -0.85963513  0.24441646]
 [-0.02435463 -0.99970128  0.00205059  0.10190226]
 [ 0.          0.          0.          1.        ]]
[[-0.85940747  0.51071093 -0.02435463 -0.00130357]
 [ 0.01988883 -0.01420518 -0.99970128  0.10254261]
 [-0.51090433 -0.85963513  0.00205059  0.28185688]
 [ 0.          0.          0.          1.        ]]


In [42]:
def transform_blocks(pcd, icp_transform):
    pcd_temp = copy.deepcopy(pcd)
    aa = o3d.cpu.pybind.utility.Vector3dVector(np.array(pcd_temp.points))
    pcd_temp.points = aa
    pcd_temp.transform(icp_transform)
    
    return pcd_temp

In [66]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'green'
target_block_label = 3

**처음 카메라 돌릴때 알아내야하는 것**
1. MESH <-> CAMERA 변환 관계
2. 각 블럭(or 색)의 초기 상태를 알면 됨
	* 어떤 방향으로 놓여있는지 (x or y)
	* 몇층에 놓여있는지 or z축 좌표

* case 1
    22.5
    52.5
    82.5
    112.5
    142.5
    172.5

* case 2
    7.5
    37.5
    67.5
    97.5
    127.5
    157.5

In [71]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    # if col != target_block_color:
    #     continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        if idx != target_block_label:
            continue
        
        # print("--------------------------------")
        print(idx)

        new_trans = copy.deepcopy(trans)
        # new_trans[0,3]*=1000
        # new_trans[1,3]*=1000
        # new_trans[2,3]*=1000

        pcd_new = transform_blocks(pcd, new_trans)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        # print("BOX CENTER Z COORDINATE : ", center_coordinate[2] * 1000)
        # print("BOX CENTER Z COORDINATE : ", center_coordinate[2] * 1000)
        # print("BOX CENTER Z COORDINATE : ", center_coordinate[2] * 1000)
        
        mok, nameoji = divmod(center_coordinate[2], 0.015)
        
        # print(mok, nameoji)
        true_z = mok * 0.015 + 0.0075
        print("true_z :", true_z)
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,0].mean()
        y_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,1].mean()
        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        
        if box_extent[1] > 0.070:
            print("################ CASE 1 ################")
            print("BLOCK DIRECTION : Y")
            print("PULL DIRECTION : X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean #- 25/2
            cen_y = y_mean #- 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[0] > 0.070:
            print("################ CASE 2 ################")
            print("BLOCK DIRECTION : X")
            print("PULL DIRECTION : Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 75/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[0]) < 0.010 and box_extent [1] < 0.015:
            print("################ CASE 3 ################")
            print("BLOCK DIRECTION : Y")
            print("PUSH DIRECTION : Y or -Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 0.075/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[1]) < 0.010 and box_extent [0] < 0.015:
            print("################ CASE 4 ################")
            print("BLOCK DIRECTION : X")
            print("PUSH DIRECTION : X or -X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 0.075/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[1] < 0.015:
            print("################ CASE 5 ################")
            print("BLOCK DIRECTION : Y")
            print("PULL DIRECTION : -X")
            vector = np.array([1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean# - 25/2
            cen_y = y_mean - 0.075/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        elif box_extent[0] < 0.015:
            print("################ CASE 6 ################")
            print("BLOCK DIRECTION : X")
            print("PULL DIRECTION : -Y")
            vector = np.array([0, 1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_mean - 0.075/2
            cen_y = y_mean# - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.00001, cen_y - 0.00001, cen_z - 0.00001], [cen_x + 0.00001, cen_y + 0.00001, cen_z + 0.00001]]))
            print("----------------------------------")
            
        else:
            print("NOTHING")
            
        if abs(cen_x) < 0.0125:
            true_x = 0.0
        elif cen_x > 0:
            true_x = 0.025
        else:
            true_x = -0.025
            
        if abs(cen_y) < 0.0125:
            true_y = 0.0
        elif cen_y > 0:
            true_y = 0.025
        else:
            true_y = -0.025
        
        print("TRUE COORDINATES")
        print("true_x :", true_x)
        print("true_y :", true_y)
        print("true_z :", true_z)
        
        pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[true_x, true_y, true_z], [true_x - 0.00001, true_y - 0.00001, true_z - 0.00001], [true_x + 0.00001, true_y + 0.00001, true_z + 0.00001]]))
            
        mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        ptc1 = o3d.geometry.PointCloud()
        ptc1.points = pt1
        ptc1.paint_uniform_color([1, 0, 0])
        ptc2 = o3d.geometry.PointCloud()
        ptc2.points = pt2
        ptc2.paint_uniform_color([0, 0, 1])
        line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        o3d.visualization.draw_geometries([pcd_new, source_temp, mesh_frame_small, ptc1, ptc2, line])

green
3
BOX EXTENT :  [0.03101941 0.07963808 0.01917698]
BOX CENTER COORDINATE :  [ 0.02756922 -0.00255576  0.06797554]
true_z : 0.0675
################ CASE 1 ################
BLOCK DIRECTION : Y
PULL DIRECTION : X
-----BLOCK CENTER COORDINATE-----
X : 0.027569219643012746
Y : -0.002555758829556734
Z : 0.06797553595676581
----------------------------------
TRUE COORDINATES
true_x : 0.025
true_y : 0.0
true_z : 0.0675
pink
3
BOX EXTENT :  [0.00709368 0.02488716 0.01619857]
BOX CENTER COORDINATE :  [ 0.03818262 -0.03424571  0.0849618 ]
true_z : 0.08249999999999999
################ CASE 6 ################
BLOCK DIRECTION : X
PULL DIRECTION : -Y
-----BLOCK CENTER COORDINATE-----
X : 0.0006826153278555944
Y : -0.03424571087042688
Z : 0.08496180444983108
----------------------------------
TRUE COORDINATES
true_x : 0.0
true_y : -0.025
true_z : 0.08249999999999999
yellow
3
BOX EXTENT :  [0.02702377 0.00602636 0.01539643]
BOX CENTER COORDINATE :  [-0.02644882  0.03943836  0.06760512]
true_z : 0

In [45]:
mesh_block_x = o3d.io.read_triangle_mesh("mesh/single_block_y_m.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()

pcd_mesh = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_mesh)

TriangleMesh with 12 points and 4 triangles.
PointCloud with 268886 points.


In [46]:
move = np.array(target_tmp.get_oriented_bounding_box().get_center() - source_tmp.get_oriented_bounding_box().get_center())

NameError: name 'target_tmp' is not defined